### 필요 라이브러리 호출

In [1]:
# library import
import os
import numpy as np
import scipy as sp
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt



In [2]:
import tensorflow as tf
import keras
from keras import optimizers
from keras import regularizers
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten, InputLayer
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV



Using TensorFlow backend.


In [3]:
print(os.path.expanduser('~'))

C:\Users\korbsl


In [4]:
path_base = os.getcwd()
path_base
# os.listdir(path_base)

'C:\\Users\\korbsl\\OneDrive - SAS\\Backup\\500_Practice\\20_Python\\1_Deep_learning\\VDMML_training'

#### 1. data import

In [6]:
file_path = 'C:\\Users\\korbsl\\OneDrive - SAS\Backup\\500_Practice\\10_SAS\\1_Deep_learning\\vdmml_data'

In [7]:
# pandas로 csv file 호출
train_develop = pd.read_csv(file_path+'\\train_develop.csv', sep=',', header=0)
valid_develop = pd.read_csv(file_path+'\\valid_develop.csv', sep=',', header=0)
test_develop  = pd.read_csv(file_path+'\\test_develop.csv', sep=',', header=0)

입력 데이터 형태 파악

In [9]:
# train, valid, test set head 출력
pd.concat([train_develop.head(2)
        ,valid_develop.head(2)
        ,test_develop.head(2)], axis=0, keys=['train','valid','test'], names=['part','row'])

AcctAge  DDA   DDABal  CashBk  Checks  DirDep  NSF  NSFAmt  Phone  \
part  row                                                                      
train 0        0.7    1  1986.81       0       1       1    0    0.00      0   
      1        4.1    0     0.00       0       0       0    0    0.00      0   
valid 0        6.7    1  2813.45       0       2       0    0    0.00      0   
      1        0.8    1   190.03       0       1       0    1    5.65      0   
test  0        0.3    1   419.27       0       0       0    0    0.00      0   
      1        0.5    1  1594.84       0       1       0    0    0.00      0   

           Teller  ...  Moved  InArea  Ins  Branch  Res  Dep   DepAmt  Inv  \
part  row          ...                                                       
train 0         0  ...      0       1    0      B2    R    1   446.93    0   
      1         0  ...      0       1    1      B3    S    0     0.00    0   
valid 0         5  ...      0       1    1      B1    S    2  1208.94    0   
      1         0  ...      0       1    0     B14    S    3   880.25    0   
test  0         0  ...      0       1    1     B17    R    2  1170.06    0   
      1         1  ...      0       1    0      B1    S    1  1144.24    0   

           InvBal  _PartInd_  
part  row                     
train 0       0.0          1  
      1       0.0          1  
valid 0       0.0          2  
      1       0.0          2  
test  0       0.0          0  
      1       0.0          0  

[6 rows x 49 columns]

variable type & final data file 생성

In [71]:
### data import code

def data_import(in_df):
    
#     path = 'C:\\beom\\_final_folder\\04_training\\01_VDMML\\1_deeplearning\\DL_VDMML_EXAM_DATA\\'
    path = file_path
    df = in_df
    
    # pandas로 csv file 호출
    df = pd.read_csv((path+'\\'+df+'.csv'), sep=',', header=0)

    # 변수 형태가 잘못 들어간 case 변경
    cat_vars = ['Branch','Res','Ins','DDA','DirDep','NSF','Sav','ATM','CD','IRA'
                    ,'LOC','ILS','MM','MTG','CC','SDB','HMOwn','Moved','InArea','Inv']
    for x in cat_vars:
        df[x] = df[x].astype(str)
    
    # 변수 유형별 리스트 셋 정의
    feature_list = [name for name in df.columns if (name !='Ins')&(name !='_PartInd_')]
    cat_vars = df.columns[(df.dtypes=='object')&(df.columns !='Ins')&(df.columns !='_PartInd_')]
    num_vars = [name for name in df.columns if (name not in cat_vars)&(name !='Ins')&(name !='_PartInd_')]

    # categorical data를 dummy로 변경
    x_dummy = pd.get_dummies(df[cat_vars], prefix_sep='_')
#     y_dummy = pd.get_dummies(df['Ins'], drop_first=True)
    y_dummy = pd.get_dummies(df['Ins'], prefix_sep='_')
    
    # numeric variable standardization
    df[num_vars] = (df[num_vars] - np.mean(df[num_vars], axis=0)) / np.std(df[num_vars], axis=0)

    # 최종 X, Y data set 생성
    out_x = np.c_[df[num_vars].values, x_dummy.values].astype(np.float64)
    out_y = y_dummy.values.ravel()
    
    # 모든 변수 이름
    feature_names = num_vars + x_dummy.columns.tolist()
    
    return out_x, out_y, feature_names

train_x = data_import(in_df='train_develop')[0]
train_y = data_import(in_df='train_develop')[1]

valid_x = data_import(in_df='valid_develop')[0]
valid_y = data_import(in_df='valid_develop')[1]

test_x = data_import(in_df='test_develop')[0]
test_y = data_import(in_df='test_develop')[1]

feature_names = data_import(in_df='train_develop')[2]

In [118]:
train_develop.Ins.value_counts()/train_develop.shape[0]

0    0.653632
1    0.346368
Name: Ins, dtype: float64

#### 2. Create Deep learning architecture 

##### 2-1. using Tensorflow

In [126]:
# 신경망 모델 구성

tf.reset_default_graph() 


X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

dropout_rate = tf.placeholder(tf.float32)


W1 = tf.get_variable("W1", shape=[len(feature_names), 30], initializer=tf.contrib.layers.xavier_initializer())
W2 = tf.get_variable("W2", shape=[30, 20], initializer=tf.contrib.layers.xavier_initializer())
W3 = tf.get_variable("W3", shape=[20, 10], initializer=tf.contrib.layers.xavier_initializer())
W4 = tf.get_variable("W4", shape=[10,  5], initializer=tf.contrib.layers.xavier_initializer())
W5 = tf.get_variable("W5", shape=[5,   1])

b1 = tf.Variable(tf.zeros([30]), name="b1")
b2 = tf.Variable(tf.zeros([20]), name="b2")
b3 = tf.Variable(tf.zeros([10]), name="b3")
b4 = tf.Variable(tf.zeros([ 5]), name="b4")
b5 = tf.Variable(tf.zeros([ 1]), name="b5")

L1 = tf.add(tf.matmul(X,W1),b1)
L1 = tf.nn.relu(L1)
L1 = tf.nn.dropout(L1, dropout_rate)

L2 = tf.add(tf.matmul(L1,W2),b2)
L2 = tf.nn.relu(L2)
L2 = tf.nn.dropout(L2, dropout_rate)

L3 = tf.add(tf.matmul(L2,W3),b3)
L3 = tf.nn.relu(L3)
L3 = tf.nn.dropout(L3, dropout_rate)

L4 = tf.add(tf.matmul(L3,W4),b4)
L4 = tf.nn.relu(L4)
L4 = tf.nn.dropout(L4, dropout_rate)

Lout = tf.add(tf.matmul(L4,W5),b5)



cost = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=Lout))

optimizer = tf.train.AdamOptimizer(learning_rate=0.01, beta1=0.9, beta2=0.999)
train_op = optimizer.minimize(cost, global_step=tf.train.get_global_step())


In [129]:
cost_summ = tf.summary.scalar("Cost", cost)

correct = tf.equal(tf.argmax(Lout, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
accuracy_summ = tf.summary.scalar("Accuracy", accuracy, ["Epoch"])

In [128]:

epochs_completed = 0
index_in_epoch = 0

# for splitting out batches of data
def next_batch(batch_size, X_data, Y_data):
    
    num_examples = X_data.shape[0]

    global X_train
    global y_train
    global index_in_epoch
    global epochs_completed

    start = index_in_epoch
    index_in_epoch += batch_size

    # when all trainig data have been already used, it is reorder randomly    
    if index_in_epoch > num_examples:
        # finished epoch
        epochs_completed += 1
        # shuffle the data
        perm = np.arange(num_examples)
        np.random.shuffle(perm)
        X_data = X_data[perm]
        Y_data = Y_data[perm]
        # start next epoch
        start = 0
        index_in_epoch = batch_size
        assert batch_size <= num_examples
    end = index_in_epoch
    return X_data[start:end], Y_data[start:end]

In [136]:
with tf.Session() as sess:

    tf.global_variables_initializer().run()
    merged = tf.summary.merge_all()
    epoch_merged = tf.summary.merge_all("Epoch")
    batch_size = int(train_develop.shape[0]/100)
    writer = tf.summary.FileWriter("./logs", sess.graph)

    for epoch in range(1):
        for step in range(batch_size):
            train, label = next_batch(100, train_x, train_y)
        
            summ, _ = sess.run([merged, train_op], feed_dict={X: train,
                                                          Y: label,
                                                          dropout_rate: 0.05})
            writer.add_summary(summ, epoch * batch_size + step)
            
        summ, _accuracy = sess.run([epoch_merged, accuracy],
                                          feed_dict={X: test_x,
                                                     Y: test_y,
                                                     dropout_rate: 0.05}) 
        writer.add_summary(summ, (epoch + 1) * batch_size)
        print ("Epoch:", (epoch + 1))
#         print ("Test Accuracy:", _accuracy)

Epoch: 1


In [70]:
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    merged = tf.summary.merge_all()
    epoch_merged = tf.summary.merge_all("Epoch")
    writer = tf.summary.FileWriter("./logs", sess.graph)
    
    batch_size = int(train_develop.shape[0]/100)
    
    for epoch in range(1):
        for step in range(batch_size):
            train = train_x.next_batch(100)
            label = train_y.next_batch(100)
            summ, _ = sess.run([merged, train_op], feed_dict={X: train,
                                                           Y: label,
                                                           dropout_rate: 0.05})
            writer.add_summary(summ, epoch * batch_size + step)
        
        summ, _accuracy = sess.run([epoch_merged, accuracy],
                                          feed_dict={X: test_x,
                                                     Y: test_y,
                                                     dropout_rate: 0.05}) 
        writer.add_summary(summ, (epoch + 1) * batch_size)
        
        print "Epoch:", (epoch + 1)
        print "Test Accuracy:", _accuracy

AttributeError: 'numpy.ndarray' object has no attribute 'next_batch'

In [ ]:
sess = tf.Session()

for step in range(2):
    sess.run(train_op, feed_dict={X: train_x, Y: train_y})

    if (step + 1) % 10 == 0:
        print(step + 1, sess.run(cost, feed_dict={X: train_x, Y: train_y}))

In [47]:
#########
# 신경망 모델 구성
######
X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

# 첫번째 가중치의 차원은 [특성, 히든 레이어의 뉴런갯수] -> [2, 10] 으로 정합니다.
W1 = tf.Variable(tf.random_uniform([len(feature_names), 10], -1., 1.))
# 두번째 가중치의 차원을 [첫번째 히든 레이어의 뉴런 갯수, 분류 갯수] -> [10, 3] 으로 정합니다.
W2 = tf.Variable(tf.random_uniform([10, 2], -1., 1.))

# 편향을 각각 각 레이어의 아웃풋 갯수로 설정합니다.
# b1 은 히든 레이어의 뉴런 갯수로, b2 는 최종 결과값 즉, 분류 갯수인 3으로 설정합니다.
b1 = tf.Variable(tf.zeros([10]))
b2 = tf.Variable(tf.zeros([2]))

# 신경망의 히든 레이어에 가중치 W1과 편향 b1을 적용합니다
L1 = tf.add(tf.matmul(X, W1), b1)
L1 = tf.nn.relu(L1)

# 최종적인 아웃풋을 계산합니다.
# 히든레이어에 두번째 가중치 W2와 편향 b2를 적용하여 3개의 출력값을 만들어냅니다.
model = tf.add(tf.matmul(L1, W2), b2)

# 텐서플로우에서 기본적으로 제공되는 크로스 엔트로피 함수를 이용해
# 복잡한 수식을 사용하지 않고도 최적화를 위한 비용 함수를 다음처럼 간단하게 적용할 수 있습니다.
cost = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=model))

optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
train_op = optimizer.minimize(cost)


In [11]:
# grid 기초 모델 정의
def grid_base_model(learning_rate=0.01, dropout_flag=True):
    input_dims = 84
    model = Sequential()
    model.add(InputLayer(input_shape=(input_dims,), name='input'))
    
    #과적합 방지를 위한 가중치와 편향에 대한 벌점항 추가
    model.add(Dense(100, activation='relu',
                   kernel_regularizer=regularizers.l2(0.01),
                   bias_regularizer=regularizers.l2(0.01),
                   name='hidden-1'))
    
#     model.add(Dense(100,activation='relu', name='hidden-1'))
    
    # 과적합 방지를 위한 dropout 추가
    if dropout_flag :
        model.add(Dropout(0.1, name='hidden-1-drop'))
        
    model.add(Dense(1, activation='sigmoid', name='output'))
    model.summary()
    
    # 최적화 방법, 손실, 평가 방법 등을 정의
    optimizer = optimizers.Adam(lr=learning_rate)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model

# 모델 구성
model = KerasClassifier(build_fn=grid_base_model, epochs=50, verbose=0)

# 초모수 그리드 정의
param_grid = dict(learning_rate=[0.01, 0.1], dropout_flag=[True, False], batch_size=[50,100])
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, return_train_score=True)

# 초모수 그리드 적합
grid_search.fit(train_x, train_y)    

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
hidden-1 (Dense)             (None, 100)               8500      
_________________________________________________________________
hidden-1-drop (Dropout)      (None, 100)               0         
_________________________________________________________________
output (Dense)               (None, 1)                 101       
Total params: 8,601
Trainable params: 8,601
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
hidden-1 (Dense)             (None, 100)               8500      
_________________________________________________________________
hidden-1-drop (Dropout)      (None, 100)               0         
_________________________________________________________________
output (

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
hidden-1 (Dense)             (None, 100)               8500      
_________________________________________________________________
hidden-1-drop (Dropout)      (None, 100)               0         
_________________________________________________________________
output (Dense)               (None, 1)                 101       
Total params: 8,601
Trainable params: 8,601
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
hidden-1 (Dense)             (None, 100)               8500      
_________________________________________________________________
hidden-1-drop (Dropout)      (None, 100)               0         
_________________________________________________________________
output (

GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x000002094F4F0A90>,
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'learning_rate': [0.01, 0.1], 'dropout_flag': [True, False], 'batch_size': [50, 100]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [12]:
#초모수 결정
grid_search.best_params_

{'batch_size': 100, 'dropout_flag': False, 'learning_rate': 0.01}

In [13]:
# 초모수 결정을 위한 근거
grid_search.cv_results_

{'mean_fit_time': array([17.97959956, 19.17578808, 17.34431871, 17.24404454, 16.60192649,
        16.37138518, 12.77171516, 13.67412146]),
 'std_fit_time': array([0.33733091, 0.1279932 , 0.44524092, 0.84612052, 1.05524248,
        0.93645988, 0.27819395, 0.29157128]),
 'mean_score_time': array([0.29645745, 0.45755259, 0.41711187, 0.46657713, 0.53709809,
        0.59558376, 0.53977728, 0.65106797]),
 'std_score_time': array([0.01023623, 0.1084696 , 0.02789525, 0.04902036, 0.04763334,
        0.12766406, 0.01379886, 0.01885385]),
 'param_batch_size': masked_array(data=[50, 50, 50, 50, 100, 100, 100, 100],
              mask=[False, False, False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_dropout_flag': masked_array(data=[True, True, False, False, True, True, False, False],
              mask=[False, False, False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_learning_rate': masked_array

In [25]:
train_x.shape[1]

84

In [28]:
#########
# 신경망 모델 구성
######
X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

# 신경망은 2차원으로 [입력층(특성), 출력층(레이블)] -> [2, 3] 으로 정합니다.
W = tf.Variable(tf.random_uniform([84, 1], -1., 1.))

# 편향을 각각 각 레이어의 아웃풋 갯수로 설정합니다.
# 편향은 아웃풋의 갯수, 즉 최종 결과값의 분류 갯수인 3으로 설정합니다.
b = tf.Variable(tf.zeros([1]))

# 신경망에 가중치 W과 편향 b을 적용합니다
L = tf.add(tf.matmul(X, W), b)
# 가중치와 편향을 이용해 계산한 결과 값에
# 텐서플로우에서 기본적으로 제공하는 활성화 함수인 ReLU 함수를 적용합니다.
L = tf.nn.relu(L)

# 마지막으로 softmax 함수를 이용하여 출력값을 사용하기 쉽게 만듭니다
# softmax 함수는 다음처럼 결과값을 전체합이 1인 확률로 만들어주는 함수입니다.
# 예) [8.04, 2.76, -6.52] -> [0.53 0.24 0.23]
model = tf.nn.softmax(L)

# 신경망을 최적화하기 위한 비용 함수를 작성합니다.
# 각 개별 결과에 대한 합을 구한 뒤 평균을 내는 방식을 사용합니다.
# 전체 합이 아닌, 개별 결과를 구한 뒤 평균을 내는 방식을 사용하기 위해 axis 옵션을 사용합니다.
# axis 옵션이 없으면 -1.09 처럼 총합인 스칼라값으로 출력됩니다.
#        Y         model         Y * tf.log(model)   reduce_sum(axis=1)
# 예) [[1 0 0]  [[0.1 0.7 0.2]  -> [[-1.0  0    0]  -> [-1.0, -0.09]
#     [0 1 0]]  [0.2 0.8 0.0]]     [ 0   -0.09 0]]
# 즉, 이것은 예측값과 실제값 사이의 확률 분포의 차이를 비용으로 계산한 것이며,
# 이것을 Cross-Entropy 라고 합니다.
cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(model), axis=1))

optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
train_op = optimizer.minimize(cost)


In [31]:

#########
# 신경망 모델 학습
######
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

for step in range(2):
    sess.run(train_op, feed_dict={X: train_x, Y: train_y})

    if (step + 1) % 10 == 0:
        print(step + 1, sess.run(cost, feed_dict={X: train_x, Y: train_y}))

In [ ]:

#########
# 결과 확인
# 0: 기타 1: 포유류, 2: 조류
######
# tf.argmax: 예측값과 실제값의 행렬에서 tf.argmax 를 이용해 가장 큰 값을 가져옵니다.
# 예) [[0 1 0] [1 0 0]] -> [1 0]
#    [[0.2 0.7 0.1] [0.9 0.1 0.]] -> [1 0]
prediction = tf.argmax(model, 1)
target = tf.argmax(Y, 1)
print('예측값:', sess.run(prediction, feed_dict={X: train_x}))
print('실제값:', sess.run(target, feed_dict={Y: train_y}))

is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도: %.2f' % sess.run(accuracy * 100, feed_dict={X: x_data, Y: y_data}))

In [16]:
#########
# 신경망 모델 구성
######
X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

# 첫번째 가중치의 차원은 [특성, 히든 레이어의 뉴런갯수] -> [2, 10] 으로 정합니다.
W1 = tf.Variable(tf.random_uniform([len(feature_names), 10], -1., 1.))
# 두번째 가중치의 차원을 [첫번째 히든 레이어의 뉴런 갯수, 분류 갯수] -> [10, 3] 으로 정합니다.
W2 = tf.Variable(tf.random_uniform([10, 2], -1., 1.))

# 편향을 각각 각 레이어의 아웃풋 갯수로 설정합니다.
# b1 은 히든 레이어의 뉴런 갯수로, b2 는 최종 결과값 즉, 분류 갯수인 3으로 설정합니다.
b1 = tf.Variable(tf.zeros([10]))
b2 = tf.Variable(tf.zeros([2]))

# 신경망의 히든 레이어에 가중치 W1과 편향 b1을 적용합니다
L1 = tf.add(tf.matmul(X, W1), b1)
L1 = tf.nn.relu(L1)

# 최종적인 아웃풋을 계산합니다.
# 히든레이어에 두번째 가중치 W2와 편향 b2를 적용하여 3개의 출력값을 만들어냅니다.
model = tf.add(tf.matmul(L1, W2), b2)

# 텐서플로우에서 기본적으로 제공되는 크로스 엔트로피 함수를 이용해
# 복잡한 수식을 사용하지 않고도 최적화를 위한 비용 함수를 다음처럼 간단하게 적용할 수 있습니다.
cost = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=model))

optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
train_op = optimizer.minimize(cost)


#########
# 신경망 모델 학습
######
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

for step in range(100):
    sess.run(train_op, feed_dict={X: train_x, Y: train_y})

    if (step + 1) % 10 == 0:
        print(step + 1, sess.run(cost, feed_dict={X: test_x, Y: test_y}))


InvalidArgumentError: logits and labels must be broadcastable: logits_size=[19358,2] labels_size=[1,19358]
	 [[node softmax_cross_entropy_with_logits_1 (defined at <ipython-input-16-ae9f2108c0c2>:28) ]]

Caused by op 'softmax_cross_entropy_with_logits_1', defined at:
  File "C:\ProgramData\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\ProgramData\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\ProgramData\Anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 505, in start
    self.io_loop.start()
  File "C:\ProgramData\Anaconda3\lib\site-packages\tornado\platform\asyncio.py", line 132, in start
    self.asyncio_loop.run_forever()
  File "C:\Prog

In [ ]:
#
# 결과 확인
# 0: 기타 1: 포유류, 2: 조류
######
prediction = tf.argmax(model, 1)
target = tf.argmax(Y, 1)
print('예측값:', sess.run(prediction, feed_dict={X: x_data}))
print('실제값:', sess.run(target, feed_dict={Y: y_data}))

is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도: %.2f' % sess.run(accuracy * 100, feed_dict={X: x_data, Y: y_data}))